# First pass look at high speed video 

For now, just want to make sure I can open videos, read images, put different views together, etc.

In [1]:
import os
import glob
import cv2 

import numpy as np

## Load data

In [2]:
# path to parent folder containing videos from multiple cameras
root_path = "/media/sam/SamData/HighSpeedVideo/SamHingeTwoCameraTest"
cameras = ["Camera_1", "Camera_2"]  # camera names are the first part of the folders containing video files
date_str = "171306"  # the particular trigger event/videos we're looking at

file_ext = ".avi"  # for now looking at avi files -- should maybe switch to mraw eventually?

In [3]:
# locate paths to the video files we care about
folder_str = os.path.join(root_path, "%s_*_%s", "*", "*%s")
video_paths = [glob.glob(folder_str%(cam, date_str, file_ext))[0] for cam in cameras]

# do a little print check 
[print(p + "\n") for p in video_paths] ;

/media/sam/SamData/HighSpeedVideo/SamHingeTwoCameraTest/Camera_1_C001H001S0001_20221122_171306/Camera_1_C001H001S0001_20221122_171306_S0001_20221129_100639/Camera_1_C001H001S0001_20221122_171306_S0001_20221129_100639.avi

/media/sam/SamData/HighSpeedVideo/SamHingeTwoCameraTest/Camera_2_C002H001S0001_20221122_171306/Camera_2_C002H001S0001_20221122_171306_S0001_20230109_131733/Camera_2_C002H001S0001_20221122_171306_S0001_20230109_131733.avi



## Basic test -- use OpenCV to read both videos and play combined frames

In [4]:
# create video capture objects for each movie
PLAY_TEST_FLAG = False
vid_caps = [cv2.VideoCapture(path) for path in video_paths]

for cap in vid_caps:
    width  = cap.get(3)  # float `width`
    height = cap.get(4)  # float `height`
    print('width = ', width)
    print('height = ', height)
    
# set 1/fp(m)s
dt = 20

# get total frame count for the movie(s)
N_frames = min([int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) for cap in vid_caps])
    
# initialize window for display
cv2.namedWindow('combined views', cv2.WINDOW_NORMAL)

# loop through frames, load images, and displayprint(folder_name)
while all([cap.isOpened() for cap in vid_caps]):
    
    # maybe we want to skip this
    if not PLAY_TEST_FLAG:
        break 
    # read both frames and boolean return values
    rets, frames = list(zip(*[cap.read() for cap in vid_caps]))
    
    # make sure we got frames
    if not all(rets):
        print("Can't read frames -- exiting")
        break

    # if we read images, convert and display them 
    imgs = [cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in frames]
    img_combined = np.hstack(tuple(imgs))
    cv2.imshow('combined views', img_combined)
    
    # press ESC to exit
    k = cv2.waitKey(dt) & 0xff
    if k == 27:
        break

# # release video captures
# [cap.release() for cap in vid_caps] ;
# cv2.destroyAllWindows()

width =  768.0
height =  768.0
width =  768.0
height =  768.0


In [5]:
[cap.release() for cap in vid_caps] ;
cv2.destroyAllWindows()

## Save new combined video? 

Uses much of the same code as the 'basic test' stuff above

In [20]:
# get path to directory wherein we can save combined video file
video_path_split = video_paths[0].split('/')
folder_name = video_path_split[-2]
folder_name_split = folder_name.split('_')
file_id = '_'.join(folder_name_split[3:])
file_ext = '.avi'

save_fn = 'combined_cameras_' + file_id + file_ext
save_path = os.path.join(root_path, 'combined', 'combined_' + file_id)
if not os.path.exists(save_path):
    os.mkdir(save_path)

save_path_full = os.path.join(save_path, save_fn)
print(save_path_full)

/media/sam/SamData/HighSpeedVideo/SamHingeTwoCameraTest/combined_20221122_171306_S0001_20221129_100639/combined_cameras_20221122_171306_S0001_20221129_100639.avi


In [21]:
# create video writing object 
# fourcc = cv2.VideoWriter_fourcc(*'MJPG')
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
FPS = int(50)
frameSize = (height, 2*width)

vid_writer = cv2.VideoWriter(save_path_full, fourcc, FPS, (int(2*width), int(height)))

In [22]:
# loop over frames and write new video file
# create video capture objects for each movie
vid_caps = [cv2.VideoCapture(path) for path in video_paths]

# get total frame count for the movie(s)
N_frames = min([int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) for cap in vid_caps])

# loop through frames, load images, and displayprint(folder_name)
while all([cap.isOpened() for cap in vid_caps]):
    # read both frames and boolean return values
    rets, frames = list(zip(*[cap.read() for cap in vid_caps]))

    # make sure we got frames
    if not all(rets):
        print("Can't read frames -- exiting")
        break

    # if we read images, convert and combine them 
    # imgs = [cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in frames]
    frame_combined = np.hstack(tuple(frames))
    # print(frame_combined.shape)
    
    # write combined image to new video file
    vid_writer.write(frame_combined)

# release video captures and writers
[cap.release() for cap in vid_caps] ;
vid_writer.release()
print('finished writing new video')

Can't read frames -- exiting
finished writing new video
